# MultiQueryRetriever

- Author: [hong-seongmin](https://github.com/hong-seongmin)
- Design: 
- Peer Review: [Hye-yoon](https://github.com/Hye-yoonJeong),[Wooseok-Jeong](https://github.com/jeong-wooseok)
- This is a part of [LangChain OpenTutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)


## Overview

`MultiQueryRetriever` offers an automated solution to enhance distance-based vector database searches by generating multiple queries from diverse perspectives using a Language Learning Model (LLM). This approach addresses the challenges of manual prompt tuning and improves the quality and comprehensiveness of search results without extensive model fine-tuning.

- **Automated Prompt Tuning**: Leverages an LLM to create multiple queries from various viewpoints based on a single user input, eliminating the need for manual adjustments and prompt engineering.
- **Comprehensive Document Retrieval**: Executes searches for each generated query and combines the unique documents from all queries, resulting in a larger and more relevant set of documents.
- **Enhanced Search Robustness**: Mitigates the limitations of distance-based searches by capturing subtle differences and deeper meanings in data, ensuring more accurate and contextually relevant search outcomes.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Usage](#usage)
- [How to use the LCEL Chain](#how-to-use-the-LCEL-Chain)

### References

- [LangChain Documentation: Few-shot prompting](https://python.langchain.com/docs/concepts/few_shot_prompting)
- [How to better prompt when doing SQL question-answering](https://python.langchain.com/docs/how_to/sql_prompting/#few-shot-examples)

---

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

    numpy (>=1.19.*) ; python_version >= "3.7"
           ~~~~~~~^


In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langchain",
        "langchain_core",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Configuration file to manage API keys as environment variables
from dotenv import load_dotenv

# Load API key information
load_dotenv()

True

In [15]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "06-Multi-Query-Retriever",
    }
)

Environment variables have been set successfully.


## Building a Vector Database with LangChain

This section demonstrates how to set up a simple vector database using LangChain components, load documents from a URL, and perform a search query.

In [6]:
# Build a sample vector DB
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load a blog post
loader = WebBaseLoader(
    "https://python.langchain.com/docs/introduction/", encoding="utf-8"
)

# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# Define embedding
openai_embedding = OpenAIEmbeddings()

# Create the vector DB
db = FAISS.from_documents(docs, openai_embedding)

# Create a retriever
retriever = db.as_retriever()

# Document search
query = "Please explain the key features and architecture of the LangChain framework."
relevant_docs = retriever.invoke(query)

# Print the number of retrieved documents
len(relevant_docs)

4

Print the content of one of the retrieved documents.

In [7]:
# Print document #1
print(relevant_docs[1].page_content)

가장 강력한 도구로서, Assistant에게 사용자 정의 함수를 지정할 수 있습니다. 이는 Chat Completions API에서의 함수 호출과 매우 유사합니다.


Function calling(함수 호출) 도구를 사용하면 Assistant 에게 사용자 정의 함수 를 설명하여 호출해야 하는 함수를 인자와 함께 지능적으로 반환하도록 할 수 있습니다.


Assistant API는 실행 중에 함수를 호출할 때 실행을 일시 중지하며, 함수 호출 결과를 다시 제공하여 Run 실행을 계속할 수 있습니다. (이는 사용자 피드백을 받아 재게할 수 있는 의미이기도 합니다. 아래 튜토리얼에서 상세히 다룹니다).


## Usage

Simply specify the LLM to be used in `MultiQueryRetriever` and pass the query, and the retriever will handle the rest.


In [8]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI


# Initialize the ChatOpenAI language model with temperature set to 0.
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

multiquery_retriever = MultiQueryRetriever.from_llm(  # Initialize the MultiQueryRetriever using the language model.
    # Pass the vector database retriever and the language model.
    retriever=db.as_retriever(),
    llm=llm,
)

Below is code that you can run to debug the intermediate process of generating multiple queries.

First, we retrieve the `"langchain.retrievers.multi_query"` logger.

This is done using the `logging.getLogger()` function. Then, we set the logger's log level to `INFO`, so that only log messages at the `INFO` level or above are printed.


In [9]:
# Logging settings for the query
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

This code uses the `invoke` method of the `retriever_from_llm` object to search for documents relevant to the given `question`.

The retrieved documents are stored in the variable `unique_docs`, and checking the length of this variable lets you see how many relevant documents were found. Through this process, you can effectively locate information related to the user's question and assess how much of it is available.


In [10]:
# Define the question
question = "Please explain the key features and architecture of the LangChain framework."
# Document search
relevant_docs = multiquery_retriever.invoke(question)

# Return the number of unique documents retrieved.
print(
    f"===============\nNumber of retrieved documents: {len(relevant_docs)}",
    end="\n===============\n",
)

# Print the content of the retrieved documents.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the steps to implement Functions using the OpenAI Assistant API?  ', '2. Can you explain the process of utilizing Functions within the OpenAI Assistant API?  ', '3. What should I know about working with Functions in the OpenAI Assistant API?']


Number of retrieved documents: 5
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용


## How to use the LCEL Chain

- Define a custom prompt, then create a Chain with that prompt.
- When the Chain receives a user question (in the following example), it generates 5 questions, and returns the 5 generated questions separated by "\n".


In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the prompt template (written to generate 5 questions)
prompt = PromptTemplate.from_template(
    """You are an AI language model assistant. 
Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. 
By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. 
Your response should be a list of values separated by new lines, eg: `foo\nbar\nbaz\n`

#ORIGINAL QUESTION: 
{question}

#Answer in Korean:
"""
)

# Create an instance of the language model.
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# Create the LLMChain.
custom_multiquery_chain = (
    {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
)

# Define the question.
question = "Please explain the key features and architecture of the LangChain framework."

# Execute the chain and check the generated multiple queries.
multi_queries = custom_multiquery_chain.invoke(question)
# Check the result (5 generated questions)
multi_queries

'OpenAI Assistant API에서 함수 사용 방법에 대해 알려주세요.\n\nOpenAI Assistant API에서 함수 활용에 대한 정보를 제공해 주세요.\n\nOpenAI Assistant API에서 함수 사용법에 대해 설명해 주세요.\n\nOpenAI Assistant API에서 함수 기능을 사용하는 방법을 알려주세요.\n\nOpenAI Assistant API에서 함수 사용에 관한 내용을 알고 싶습니다.'

You can pass the previously created Chain to `MultiQueryRetriever` to perform retrieval.

In [12]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=custom_multiquery_chain, retriever=db.as_retriever()
)

Use `MultiQueryRetriever` to search documents and check the results.

In [13]:
# Result
relevant_docs = multiquery_retriever.invoke(question)

# Return the number of unique documents retrieved.
print(
    f"===============\nNumber of retrieved documents: {len(relevant_docs)}",
    end="\n===============\n",
)

# Print the content of the retrieved documents.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['OpenAI Assistant API에서 함수 사용 방법에 대해 알려주세요.  ', 'OpenAI Assistant API에서 함수 활용에 대한 정보를 제공해 주세요.  ', 'OpenAI Assistant API에서 함수 사용법에 대해 설명해 주실 수 있나요?  ', 'OpenAI Assistant API의 함수 사용에 대해 자세히 알고 싶습니다.  ', 'OpenAI Assistant API에서 함수 기능을 사용하는 방법은 무엇인가요?']


Number of retrieved documents: 5
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용
